In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine 

In [3]:
db_config = {
    'user': 'praktikum_student', # имя пользователя
    'pwd': 'Sdf4$2;d-d30pp', # пароль
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432, # порт подключения
    'db': 'data-analyst-advanced-sql' # название базы данных
}  

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [4]:
engine = create_engine(connection_string)

In [7]:
# пишем запрос
query = '''
 
WITH events as -- пользователь и даты его постов усеченные до месяца
    (SELECT user_id, DATE_TRUNC('month', creation_date)::date event_time
    FROM stackoverflow.posts
    GROUP BY 1,2),

     profile as --пользователь, первый пост и количество пользователей с первым постом в эту дату.
    (SELECT user_id, first_post, COUNT(*) OVER (PARTITION BY first_post) AS cohort_users_cnt
 FROM (SELECT user_id, MIN(DATE_TRUNC('month', creation_date)::date) first_post
    FROM stackoverflow.posts
    GROUP BY 1
    ORDER BY 2) as ab)
    
SELECT 
    first_post AS cohort_dt, -- дата первого поста
    event_time, -- дата события 
    COUNT(p.user_id) AS users_cnt, --количетво юзеров, у которых были события с таким же первым постом и событием
    cohort_users_cnt,
    ROUND(COUNT(p.user_id) * 100.0 / cohort_users_cnt, 2) AS retention_rate
FROM profile p 
JOIN events e ON p.user_id = e.user_id
GROUP BY 1,
         2,
         4;  
'''

sample_df = pd.read_sql_query(query, con=engine)

sample_df
# выполняем запрос

,cohort_dt,event_time,users_cnt,cohort_users_cnt,retention_rate
0,2008-07-01,2008-07-01,3,3,100.00
1,2008-07-01,2008-08-01,2,3,66.67
2,2008-07-01,2008-09-01,1,3,33.33
3,2008-07-01,2008-10-01,2,3,66.67
4,2008-07-01,2008-11-01,1,3,33.33
5,2008-07-01,2008-12-01,2,3,66.67
6,2008-08-01,2008-08-01,2151,2151,100.00
7,2008-08-01,2008-09-01,1571,2151,73.04
8,2008-08-01,2008-10-01,1275,2151,59.27
9,2008-08-01,2008-11-01,1050,2151,48.81
